In [34]:
from dotenv import load_dotenv
import os

load_dotenv(override=True)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_EMBEDDING_MODEL = os.getenv("OPENAI_EMBEDDING_MODEL")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
PINECONE_NAMESPACE = os.getenv("PINECONE_NAMESPACE")

In [35]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader('data\Past_news.csv')
docs = loader.load()
for i, d in enumerate(docs[:2]):
    print(i, d)

0 page_content='﻿ID: PN_001
Issue_name: 메르스 사태 확산
Contents: 2015년 5월부터 국내에서 중동호흡기증후군(MERS)이 확산되며 국민 불안과 소비 위축, 관광객 감소가 동반되었음. 내수 및 서비스업 중심으로 경제 전반에 충격이 가해짐.
Contentes(Spec): 확진자 및 격리자가 급증하면서 국내 유통, 외식, 여행업종이 직격탄을 맞았고, 반면 진단키트 및 백신 관련 기업은 수혜를 입음. 정부는 소비 쿠폰 등으로 대응에 나섰음.
Start_date: 2015-05-01
Fin_date: 2015-07-01
근거자료: 질병관리청 / KRX 시장동향' metadata={'source': 'data\\Past_news.csv', 'row': 0}
1 page_content='﻿ID: PN_002
Issue_name: 중국 증시 폭락 및 위안화 평가절하
Contents: 2015년 중반 이후 중국 증시 급락과 위안화 절하로 글로벌 금융시장이 충격
Contentes(Spec): 2015년 6월부터 시작된 중국 증시의 급락은 8월 중순 인민은행의 위안화 평가절하 발표와 맞물려 전 세계 금융시장의 변동성을 키웠다. 특히 원자재 수요 둔화 우려와 신흥국 경기 둔화 가능성으로 인해 글로벌 수출 의존도가 높은 한국 기업들의 실적 부진 우려가 확산되었고, 코스피도 이에 따라 큰 폭 하락하였다. 외국인 자금 유출과 함께 원화 약세가 심화되며 환율과 수출 관련 종목 중심으로 주가 조정이 나타났다.
Start_date: 2015-06-01
Fin_date: 2015-09-01
근거자료: KRX Market 2015년 하반기, BoK 금융안정보고서' metadata={'source': 'data\\Past_news.csv', 'row': 1}


In [36]:

len(docs)   # csv 행이 1205개 있다는 것

28

In [37]:
# embedding 모델 객체 생성
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")


In [38]:
from pinecone import Pinecone, ServerlessSpec

# Pinecone 클라이언트를 초기화(객체생성)
pc = Pinecone(api_key=PINECONE_API_KEY)

In [39]:
# split하기
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 텍스트 분할기 설정 (예: 1000자씩 분할)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=480, 
    chunk_overlap=48,
    # length_function=tiktoken_len,  # 토큰 기반 길이 측정    
    length_function=len,  # 문자수   
    separators=["\n\n", "\n", " ", ""]
    )

# 문서를 분할
chunks = text_splitter.split_documents(docs)

In [40]:
len(chunks)

28

In [41]:
# vector sotre에 저장
from langchain_pinecone import PineconeVectorStore

batch_size = 30
  # 한 번에 처리할 문서 수

for i in range(0, len(chunks), batch_size):
    batch_docs = chunks[i:i+batch_size]
    
    if i == 0:
        # 첫 번째 배치로 벡터 스토어 생성
        vector_store = PineconeVectorStore.from_documents(
            batch_docs,            # batch_size 수 만큼의 chunk
            embedding=embeddings,  # 임베딩 벡터로 변환
            index_name="lastproject",   # index 이름
            namespace="past_issue"    
        )
    else:
        # 이후 배치는 기존 벡터 스토어에 추가, # 내부적으로 임베딩 벡터로 변환
        vector_store.add_documents(batch_docs)  
    
    print(f"배치 {i//batch_size + 1} 완료: {len(batch_docs)}개 문서 업로드")

배치 1 완료: 28개 문서 업로드
